<font size=8>&nbsp;<font color=#3F7DCB>**q**<font color=#C9E9FF>**Bitlab =**</font> <font color=#F9AB00>**Colab**<font color=#C9E9FF> **+**<font color=#202229> **qBittorrent-nox**<font color=#C9E9FF> **+** <font color=#3FB27F>**VueTorrent**

In case you stumbled upon this colab notebook somewhere on the internet, here's the **[repo](https://github.com/hyPnOtICDo0g/Otakulab)**.

## Fetch Dependencies
Nothing fancy here. Double click below if interested. (Run this cell before anything below) 👀

In [ ]:
#@markdown #### Dependencies
%%shell
if [ -e /usr/bin/qbittorrent-nox ] && [ -e /content/vuetorrent.zip ]
then
  echo Done!
else
  pip install pyngrok
  apt update -qq -y && yes "" | add-apt-repository ppa:qbittorrent-team/qbittorrent-stable
  apt install screen qbittorrent-nox -qq
  npm install -g npm localtunnel
    
  BASE_URL="https://github.com/WDaan/VueTorrent"
  LATEST_TAG=$(curl -s "${BASE_URL}/releases/latest" | sed -En 's/.+\/tag\/([.0-9A-Za-z]+)".+/\1/p')
  wget -q  --show-progress $BASE_URL"/releases/download/"$LATEST_TAG"/vuetorrent.zip"
  unzip -qq vuetorrent.zip
  echo Done!
fi

## Mount Google Drive
To Download files we need to mount Google Drive.


In [ ]:
#@markdown #### 🔼 This cell will mount your Google Drive to /content/drive
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/1/12/Google_Drive_icon_%282020%29.svg/672px-Google_Drive_icon_%282020%29.svg.png' height="50"></center>
#@markdown #### • Type the name of the Shared/Team drive if you have one (Optional)

from google.colab import drive
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]

#Option to directly download to your team drive so as to remove the file size limits
#Names are case-sensitive
TD = ""  #@param {type:"string"}

#Enter your authorization code & click enter
if MODE == "MOUNT":
  drive.mount('/content/drive')
elif MODE == "UNMOUNT":
   drive.flush_and_unmount()

## Start a Local server & Port forward it using [*localtunnel*](https://theboroer.github.io/localtunnel-www) or [*ngrok*](https://ngrok.com)
WebUI & Colab VM act as frontend & backend.

In [ ]:
#@markdown <br><center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/66/New_qBittorrent_Logo.svg/2048px-New_qBittorrent_Logo.svg.png" height="80"></a></center><br>
#@markdown #### • Username: `qbitcolab` (Just click login after typing it)
#@markdown #### • localtunnel will be set as default as ngrok is rate limiting requests.
#@markdown #### • To change the download location or tunnel, choose the required options & run the cell again.

!echo "Configuring qBittorrent..."

#check whether team drive is initialised or not
try:
  TD
except NameError as notini:
  TD=""

from pyngrok import ngrok
from google.colab import output
import os, time
!rm -f root/.config/qBittorrent/qBittorrent.conf screenlog.0

Temp_storage_VM = True #@param {type:"boolean"}
tunnel = "localtunnel" #@param ["localtunnel", "ngrok"]
DL_LOC = "MY DRIVE (15GB)" #@param ["COLAB VM (70GB)", "MY DRIVE (15GB)", "SHARED DRIVE (∞)"]
DIR = "qBittorrent"  #@param {type:"string"}
theme = "VueTorrent" #@param ["Default", "VueTorrent"]

!mkdir -p /{content/qBittorrent,root/{.qBittorrent_temp,.config/qBittorrent}}
!curl -s https://pastebin.com/raw/n490aNnH -o /root/.config/qBittorrent/qBittorrent.conf

if theme == "Default":
  !sed -i 's|AlternativeUIEnabled=true|AlternativeUIEnabled=false|' /root/.config/qBittorrent/qBittorrent.conf

#Configuring the appropriate folder to download
if (DL_LOC == "MY DRIVE (15GB)") and os.path.isdir("/content/drive/MyDrive") == True:

#disabling this will slow down download speeds
#Since it'll be using your drive as temp storage
  if Temp_storage_VM == False:
    !sed -i 's|TempPath=/content/qBittorrent|TempPath=/content/drive/MyDrive/qBittorrent|' /root/.config/qBittorrent/qBittorrent.conf
  !sed -i 's|content/Downloads/qBittorrent|content/drive/MyDrive/'$DIR'|' /root/.config/qBittorrent/qBittorrent.conf

elif (DL_LOC == "SHARED DRIVE (\u221E)" and TD != ""):
  if Temp_storage_VM == False:
    !sed -i 's|TempPath=/content/qBittorrent|TempPath=/content/drive/Shareddrives/'$TD'/qBittorrent|' /root/.config/qBittorrent/qBittorrent.conf
  !sed -i 's|content/Downloads/qBittorrent|content/drive/Shareddrives/'$TD'/'$DIR'|' /root/.config/qBittorrent/qBittorrent.conf

else:
  !echo ""
  !echo "Google drive/Shared drive mount not detected, defaulting to Colab VM as the download location."
  !sed -i 's|content/Downloads/qBittorrent|content/Downloads/'$DIR'|' /root/.config/qBittorrent/qBittorrent.conf

#Killing the server to reload a new config
#Downloaded stuff won't be deleted & partially downloaded files will be rehashed

!pkill -9 node
!pkill -9 qbittorrent
ngrok.kill()
time.sleep(3)

#Start qbittorrent-nox as a daemon
!qbittorrent-nox -d --webui-port=3000

#Start localtunnel & open a HTTPS tunnel on port 3000
if tunnel == "localtunnel":
  !screen -dm -L lt -p 3000
  time.sleep(10)
  output.clear()
  !echo "WebUI: "$(sed -r 's/your url is: //' screenlog.0 | grep loca)

#Using pyngrok to open a HTTPS tunnel on port 3000
elif tunnel == "ngrok":
  AUTH_TOKEN = ""
  ngrok.set_auth_token(AUTH_TOKEN)
  output.clear()
  public_url = ngrok.connect(3000, "http", options={"bind_tls": True})
  !printf "WebUI: "; printf "$public_url" | grep -Eo 'http://[^ >]+' | head -1 | sed 's/http/https/g';

## Transfer downloaded files using rclone 
Run this cell only if Colab VM was set as the download location.

In [ ]:
#@markdown #### • Enter the source & destination paths & run this cell.
#@markdown #### • Upload your `rclone.conf` file when prompted.
#@markdown #### • You can get the _source path_ by using the file manager to your left.
#@markdown #### • _Destination path_ is the remote you wanna transfer to.

from os import path
if DL_LOC == "COLAB VM (70GB)":
  from google.colab import files
  if path.exists("/root/.config/rclone/rclone.conf") == False:

    files.upload()
    !mkdir -p /root/.config/rclone;mv rclone.conf /root/.config/rclone
    !curl https://rclone.org/install.sh | sudo bash
    output.clear()

  Source = "" #@param {type:"string"}
  Destination = "" #@param {type:"string"}

  !echo "Transferring..."
  !rclone copy "$Source" "$Destination"
  output.clear()
  !echo "Done!"

## Kill the server ☠️

*Omae wa mou shinderu.*

In [ ]:
#@markdown (× _ ×;)
#Change status to ded to kill the server
%%shell
STATUS="aLivE" #@param ["aLivE", "ded"]
if [ "$STATUS" = "ded" ]
then 
  pkill -9 ngrok;pkill -9 node;pkill -9 qbittorrent
  printf "Well, Done & Dusted."
fi